## Protocol 3 : Evaluating the Performance of Visual Stimulus Presentation

This protocol aims to verify the temporal accuracy and stability of the actual appearance of a visual stimulus on a screen.

### Prerequisites

**Hardware:**
* A digital oscilloscope or acquisition device (here Analog Discovery 3)
* **Two computer monitors**
* A **photodiode** with its power supply circuit

**Software:**
* Acquisition and measurement software (here **Digilent Waveforms**, version `3.24.2`)
* **Python 3** with additional libraries (e.g., `pyserial`, `expyriment`, `psychopy2`)

### Procedure

#### 1. Stimulus Generation

1. **Set up two screens**: one for the recording window (Waveforms) and the other for the visual stimulus display.

2. **Prepare your `.py` code** that displays visual stimuli, using the desired period as a parameter.

> **Python (.py) code to display stimuli**
> ```python
> from expyriment import design, control, stimuli, misc
>
> # --- Setup Expyriment ---
> exp = design.Experiment(name="Stimuli_Only")
> control.set_develop_mode(True)
> control.initialize(exp)
>
> PERIOD = 250  # Total interval in ms
> SQUARE_DURATION = 100  # ms
>
> square = stimuli.Rectangle((400, 400), position=(0, 0))
> blank = stimuli.BlankScreen()
> square.preload()
> blank.preload()
>
> exp.add_data_variable_names(['trial', 'stimulus_time'])
>
> control.start(skip_ready_screen=True)
> clock2 = misc.Clock()
> i = 1
>
> while i <= 10000:
>     while (clock2.time < i * PERIOD - 3):
>         pass
>
>     stim_time = clock2.time + square.present(update=True)
>
>     while (clock2.time - stim_time < SQUARE_DURATION - 3):
>         pass
>     blank.present(update=True)
>
>     exp.data.add([i, stim_time])
>     i += 1
>     exp.keyboard.process_control_keys()
>
> control.end()
> ```

3. **Run the `.py` script** in your terminal.  
> ![Scope settings](../pictures/expy_stimuli_only.png)

4. A window should now display the stimuli. **Move this window to the secondary screen**.

---

#### 2. Signal Measurement

1. **Connect the oscilloscope probes:**
   * GND of the oscilloscope to the photodiode's ground.
   * Measurement probe to the output (Vout) of the photodiode.

2. **Place the photodiode** over the window where the stimuli are displayed.

3. **Launch the Waveforms software** (or equivalent). Click on **`Scope`**  
> ![Scope settings](../pictures/welcome_page_waveforms.png)  
> ![Scope settings](../pictures/scope_opened.png)

4. Click **`Run`** to start acquisition. You should observe a signal that varies with the light intensity received by the photodiode.  
> ![Scope settings](../pictures/acquisition_photodiode.png)

5. You can visually inspect the signal (period, amplitude, duty cycle) by **adjusting the axis scale**.  
> ![Scope settings](../pictures/acquisition_characteristics_one_channel.png)

6. **Adjust the scales** and start acquisition with the **`Run`** button.

---

#### 3. Recording Signals

1. Click the **`Record`** button.  
> ![Scope settings](../pictures/record_button.png)

2. Set the recording parameters:
   * **File format:** `.wav`
   * **Sampling rate:** `10 kHz`
   * **Duration:** `20 s`
   * **File location and name**  
> ![Scope settings](../pictures/choice_of_recording_characteristics_wav.png)

3. Start the recording. You now have a `.wav` file containing the measured signal.

---

#### 4. Signal Analysis

1. **Prepare the Python script.**  
   The code should read the `.wav` file, apply a detection threshold, a minimum interval between peaks, and compute min/max/mean intervals, generate a histogram, and mark detected peaks.

> **Python (.py) code for photodiode signal analysis**
> ```python
> import numpy as np
> import matplotlib
> matplotlib.use('Agg')  # Non-interactive backend
> import matplotlib.pyplot as plt
> from scipy.io import wavfile
> from scipy.signal import find_peaks
> import os
>
> # --- Parameters ---
> NOM_FICHIER_WAV = 'psychopy_stimuli_only_250ms.wav'
> DISTANCE_MIN_MS = 100.0
>
> def analyser_et_sauvegarder_graphique(nom_fichier):
>     try:
>         sampling_rate, data = wavfile.read(nom_fichier)
>         print(f"\n--- File analysis: {nom_fichier} ---")
>         print(f"Sampling rate: {sampling_rate} Hz")
>
>         if data.ndim > 1:
>             data = data[:, 0]
>
>         signal_abs = np.abs(data)
>         seuil = 0.8 * np.max(signal_abs)
>         print(f"Detection threshold (80% max amplitude): {int(seuil)}")
>
>         distance_min_samples = int(DISTANCE_MIN_MS * sampling_rate / 1000)
>
>         indices_pics, _ = find_peaks(signal_abs, height=seuil, distance=distance_min_samples)
>         intervalles_en_samples = np.diff(indices_pics)
>         intervalles_en_ms = (intervalles_en_samples / sampling_rate) * 1000
>
>         print(f"\nDetected peaks: {len(indices_pics)}")
>         print(f"Intervals: {len(intervalles_en_ms)}")
>
>         if len(intervalles_en_ms) > 0:
>             moyenne = np.mean(intervalles_en_ms)
>             print(f"Mean: {moyenne:.2f} ms")
>             print(f"Min: {np.min(intervalles_en_ms):.2f} ms")
>             print(f"Max: {np.max(intervalles_en_ms):.2f} ms")
>
>             # --- Histogram ---
>             plt.figure(figsize=(12, 6))
>             plt.hist(intervalles_en_ms, bins=50, edgecolor='black', alpha=0.75)
>             plt.axvline(moyenne, color='red', linestyle='--', linewidth=2, label=f'Mean = {moyenne:.2f} ms')
>             plt.title(f"Interval Distribution — {os.path.basename(nom_fichier)}", fontsize=14)
>             plt.xlabel("Interval duration (ms)")
>             plt.ylabel("Number of intervals")
>             plt.legend()
>             plt.grid(True, linestyle='--', alpha=0.6)
>
>             output_dir = 'figures'
>             os.makedirs(output_dir, exist_ok=True)
>             base_name = os.path.splitext(os.path.basename(nom_fichier))[0]
>             save_hist_path = os.path.join(output_dir, f'distribution_intervalles_{base_name}.png')
>             plt.savefig(save_hist_path, dpi=300, bbox_inches='tight')
>             plt.close()
>             print(f"\nHistogram saved: {save_hist_path}")
>
>             # --- Signal with peaks ---
>             plt.figure(figsize=(15, 5))
>             plt.plot(data, label='Signal')
>             plt.plot(indices_pics, data[indices_pics], 'rx', label='Detected peaks')
>             plt.title(f"Signal and Detected Peaks — {base_name}")
>             plt.xlabel("Samples")
>             plt.ylabel("Amplitude")
>             plt.legend()
>             save_signal_path = os.path.join(output_dir, f'signal_pics_{base_name}.png')
>             plt.savefig(save_signal_path, dpi=300, bbox_inches='tight')
>             plt.close()
>             print(f"Signal plot saved: {save_signal_path}")
>
>         else:
>             print("No intervals detected — no figure generated.")
>
>     except FileNotFoundError:
>         print(f"ERROR: File '{nom_fichier}' not found.")
>     except Exception as e:
>         print(f"An error occurred: {e}")
>
> if __name__ == '__main__':
>     analyser_et_sauvegarder_graphique(NOM_FICHIER_WAV)
> ```

2. **Run the script.**

3. You will see in the terminal the signal and latency interval characteristics, and the histogram will be saved to the specified folder.  
> ![Scope settings](../pictures/command_results_photodiode.png)  
> ![Scope settings](../pictures/results_plot_photodiode.png)